In [133]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
from sklearn_utils import *
from tensorflow_utils import *

import numpy as np
import pandas as pd
import tensorflow as tf
import nltk
import sklearn
from sklearn.cross_validation import train_test_split
from sklearn.pipeline import Pipeline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [111]:
data_filename  = '../data/train.csv'

data_df = pd.read_csv(data_filename)
corpus = data_df['Comment']
labels = data_df['Insult']
train_corpus, test_corpus, train_labels, test_labels = \
sklearn.cross_validation.train_test_split(corpus, labels, test_size=0.33)

In [112]:
vecorizer = Pipeline(
    [
        ('vect', sklearn.feature_extraction.text.CountVectorizer(
                lowercase=False,
                analyzer='char',
                ngram_range=(1,4),
                )
        ),
        ('tfidf', sklearn.feature_extraction.text.TfidfTransformer(
                sublinear_tf=True,norm='l2')
        ),   
    ]).fit(train_corpus)

In [113]:
X_train = vecorizer.transform(train_corpus)
y_train = train_labels.values

In [114]:
X_test = vecorizer.transform(test_corpus)
y_test = test_labels.values

## Sklearn Logisitc Regression and Linar SVM

In [128]:
alg = sklearn.linear_model.LogisticRegression()
param_grid = {'C': [0.1, 1, 5,  50, 100, 1000, 5000]}
model = cv (X_train, y_train, 5, alg, param_grid, 'roc_auc', False)




Doing Gridsearch

Best Model Train AUC: 0.999748
Best Model Train F1 0.968935
Best Model Train Accuracy 0.984115

Best parameters set found:
{'C': 5} 0.901330441002


Grid scores:
0.85274 (+/-0.00600) for {'C': 0.1}
0.89049 (+/-0.00435) for {'C': 1}
0.90133 (+/-0.00340) for {'C': 5}
0.90083 (+/-0.00284) for {'C': 50}
0.90013 (+/-0.00286) for {'C': 100}
0.89770 (+/-0.00282) for {'C': 1000}
0.89648 (+/-0.00279) for {'C': 5000}


In [126]:
alg = sklearn.svm.LinearSVC()
param_grid = {'C': [5, 10, 20, 50, 100, 150, 1000, 5000]}
model = cv (X_train, y_train, 5, alg, param_grid, 'roc_auc', False)




Doing Gridsearch

Best Model Train AUC: 1.000000
Best Model Train F1 1.000000
Best Model Train Accuracy 1.000000

Best parameters set found:
{'C': 5} 0.894533198307


Grid scores:
0.89453 (+/-0.00856) for {'C': 5}
0.89277 (+/-0.00860) for {'C': 10}
0.89145 (+/-0.00865) for {'C': 20}
0.89073 (+/-0.00864) for {'C': 50}
0.89050 (+/-0.00865) for {'C': 100}
0.89043 (+/-0.00865) for {'C': 150}
0.89029 (+/-0.00864) for {'C': 1000}
0.89027 (+/-0.00864) for {'C': 5000}


In [129]:
model, metrics = evaluate(
    X_train,
    y_train,
    sklearn.linear_model.LogisticRegression(C=5),
    False,
    verbose = True,
    X_test = X_test,
    y_test = y_test
)

# Note this score is lower than the cv scores since we "overfit" during cv by not retraining
# the transformer on each fold

{'AUC': {'test': 0.88070103105015241, 'train': 0.99974778908658979},
 'Accuracy': {'test': 0.8135072908672295, 'train': 0.98411497730711039},
 'F1': {'test': 0.58883248730964466, 'train': 0.96893491124260356}}


## Tensorflow MPL

In [140]:
isinstance(y_train, np.ndarray)

True

In [145]:
y_train = np.array([y_train, 1- y_train]).T
y_test = np.array([y_test, 1- y_test]).T

In [157]:
# Parameters
learning_rate = 0.001
training_epochs = 60
batch_size = 200
display_step = 5

# Network Parameters
n_hidden_1 = 100 # 1st layer num features
n_hidden_2 = 100 # 2nd layer num features
n_hidden_3 = 100 # 2nd layer num features
n_input = X_train.shape[1]
n_classes = 2 

# tf Graph input
x = tf.placeholder("float", [None, n_input])
y = tf.placeholder("float", [None, n_classes])

# Create model
def LG(_X, _weights, _biases):
    return tf.matmul(_X, _weights['out']) + _biases['out']

# Store layers weight & bias
weights = {
    'out': tf.Variable(tf.random_normal([n_input, n_classes]))
}
biases = {
    'out': tf.Variable(tf.random_normal([n_classes]))
}

# Construct model
pred = LG(x, weights, biases)

# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(pred, y)) # Softmax loss
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost) # Adam Optimizer

# Initializing the variables
init = tf.initialize_all_variables()

# Launch the graph

sess = tf.Session()

sess.run(init)

# Training cycle
for epoch in range(training_epochs):
    avg_cost = 0.
    m = 0
    batches = batch_iter(X_train.toarray(), y_train, batch_size)
    # Loop over all batches
    for batch_xs, batch_ys in batches:
        batch_m = len(batch_ys)
        m += batch_m
        # Fit training using batch data
        sess.run(optimizer, feed_dict={x: batch_xs, y: batch_ys})
        # Compute average loss
        avg_cost += sess.run(cost, feed_dict={x: batch_xs, y: batch_ys}) * batch_m
    # Display logs per epoch step
    if epoch % display_step == 0:
        print ("Epoch:", '%04d' % (epoch+1), "cost=", "{:.9f}".format(avg_cost/m))
        correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
        # Calculate accuracy
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
        print ("Accuracy:", accuracy.eval({x: X_train.toarray(), y: y_train}, session=sess))
        print ("Accuracy:", accuracy.eval({x: X_test.toarray(), y: y_test}, session=sess))    
        
        
print ("Optimization Finished!")
# Test model


Epoch: 0001 cost= 0.858585700
Accuracy: 0.542738
Accuracy: 0.53185
Epoch: 0006 cost= 0.620715947
Accuracy: 0.681921
Accuracy: 0.653108
Epoch: 0011 cost= 0.538689288
Accuracy: 0.732602
Accuracy: 0.692249
Epoch: 0016 cost= 0.474780355
Accuracy: 0.77534
Accuracy: 0.702993
Epoch: 0021 cost= 0.420612711
Accuracy: 0.805598
Accuracy: 0.71604
Epoch: 0026 cost= 0.374425979
Accuracy: 0.831316
Accuracy: 0.72218
Epoch: 0031 cost= 0.335023997
Accuracy: 0.860439
Accuracy: 0.728319
Epoch: 0036 cost= 0.300916263
Accuracy: 0.88351
Accuracy: 0.729087
Epoch: 0041 cost= 0.271585224
Accuracy: 0.904312
Accuracy: 0.738296
Epoch: 0046 cost= 0.246153700
Accuracy: 0.92171
Accuracy: 0.740599
Epoch: 0051 cost= 0.223596772
Accuracy: 0.934947
Accuracy: 0.747506
Epoch: 0056 cost= 0.204010803
Accuracy: 0.944024
Accuracy: 0.750576
Optimization Finished!
